# Colab output wrapper

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# wrap the output in colab cells
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

# Install Transformers

In [ ]:
# install transformers with sentencepiece
!pip install transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 7.6 MB/s 
     |████████████████████████████████| 6.6 MB 53.6 MB/s 
     |████████████████████████████████| 120 kB 62.9 MB/s 
     |████████████████████████████████| 1.3 MB 50.6 MB/s 


# Read input file from Google Drive

In [ ]:
# open and read the file from google drive
file = open("001.txt", "r")
FileContent = file.read().strip()

In [ ]:
# display file content
FileContent 

'Labour plans maternity pay rise\n\nMaternity pay for new mothers is to rise by £1,400 as part of new proposals announced by the Trade and Industry Secretary Patricia Hewitt.\n\nIt would mean paid leave would be increased to nine months by 2007, Ms Hewitt told GMTV\'s Sunday programme. Other plans include letting maternity pay be given to fathers and extending rights to parents of older children. The Tories dismissed the maternity pay plan as "desperate", while the Liberal Democrats said it was misdirected.\n\nMs Hewitt said: "We have already doubled the length of maternity pay, it was 13 weeks when we were elected, we have already taken it up to 26 weeks. "We are going to extend the pay to nine months by 2007 and the aim is to get it right up to the full 12 months by the end of the next Parliament." She said new mothers were already entitled to 12 months leave, but that many women could not take it as only six of those months were paid. "We have made a firm commitment. We will definit

In [ ]:
# total characters in the file
len(FileContent) 

2597

# Load the Model and Tokenizer

In [ ]:
# import and initialize the tokenizer and model from the checkpoint
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

checkpoint = "sshleifer/distilbart-cnn-12-6"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

# Some model statistics

In [ ]:
# max tokens including the special tokens
tokenizer.model_max_length 

1024

In [ ]:
# max tokens excluding the special tokens
tokenizer.max_len_single_sentence 

1022

In [ ]:
# number of special tokens
tokenizer.num_special_tokens_to_add() 

2

# Convert file content to sentences

In [ ]:
# extract the sentences from the document
import nltk
nltk.download('punkt')
sentences = nltk.tokenize.sent_tokenize(FileContent)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# find the max tokens in the longest sentence
max([len(tokenizer.tokenize(sentence)) for sentence in sentences])

47

# Create the chunks

In [ ]:
# initialize
length = 0
chunk = ""
chunks = []
count = -1
for sentence in sentences:
  count += 1
  combined_length = len(tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter

  if combined_length  <= tokenizer.max_len_single_sentence: # if it doesn't exceed
    chunk += sentence + " " # add the sentence to the chunk
    length = combined_length # update the length counter

    # if it is the last sentence
    if count == len(sentences) - 1:
      chunks.append(chunk.strip()) # save the chunk
    
  else: 
    chunks.append(chunk.strip()) # save the chunk
    
    # reset 
    length = 0 
    chunk = ""

    # take care of the overflow sentence
    chunk += sentence + " "
    length = len(tokenizer.tokenize(sentence))
len(chunks)

1

# Some checks

In [ ]:
[len(tokenizer.tokenize(c)) for c in chunks]

[533]

In [ ]:
[len(tokenizer(c).input_ids) for c in chunks]

[535]

## With special tokens added

In [ ]:
sum([len(tokenizer(c).input_ids) for c in chunks])

535

In [ ]:
len(tokenizer(FileContent).input_ids)

543

## Without special tokens added

In [ ]:
sum([len(tokenizer.tokenize(c)) for c in chunks])

533

In [ ]:
len(tokenizer.tokenize(FileContent))

541

# Get the inputs

In [ ]:
# inputs to the model
inputs = [tokenizer(chunk, return_tensors="pt") for chunk in chunks]

# Output

In [ ]:
for input in inputs:
  output = model.generate(**input)
  print(tokenizer.decode(*output, skip_special_tokens=True))

 Labour plans maternity pay rise by £1,400 as part of new proposals announced by the Trade and Industry Secretary Patricia Hewitt. Other plans include letting maternity pay be given to fathers and extending rights to parents of older children. Tories dismiss plans as 'desperate bid to win back women voters'
